# Hey, thanks for being here!
## Capstone Project - Week 4
I know this is the final assignment but I don't feel too creative today, so I'll stick to the basic 2nd option, mentioned in the Assignment description:

*"In a city of your choice, if someone is looking to open a restaurant, where would you recommend that they open it? Similarly, if a contractor is trying to start their own business, where would you recommend that they setup their office?"*

So I will pick Budapest, as that is my hometown and I will suggest neighborhoods for the entrepreneur. He/she wants to __start a gym__, because fitness is very trendy nowadays and I would not blame him to do that, because you can join several services like massage, dietary service, consulting on it.
Therefore I will map Budapest for possible starting points for his/her new franchise in the fitness industry.

The main idea is to query the Foursquare API for gyms in Budapest (or services like so), then pick the least dense area possible to minimize the concurrence. Of course that is not a quick lifehack or recipe for success but something to start with. Also we can review their average ratings, in that case, lower is better - I bet ppl are more open to new opportunities if the gym ratings are not that good.

## Capstone Project - Week 5
Let's get to work...

In [61]:
# Let's import everything we might need throughout our expedition :)

import requests 
import pandas as pd 
import numpy as np 
import random 
from geopy.geocoders import Nominatim 
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
import folium

In [62]:
# I'm sorry but I cannot display these for you
# But don't think I did not use them!

CLIENT_ID = 'VPDI2D3XEB2I14NKL1EBV0RKHHK0Q4ZPPRJZIOXY2GFARGVV' # your Foursquare ID
CLIENT_SECRET = 'QJKHM15503UBRSR5FYC42D0EZOMSK5Q3WTIUVSSVKCMZHKLD' # your Foursquare Secret
VERSION = '20190625'
LIMIT = 400

Let's find Budapest's Longitude and Latitude values to start with.

In [63]:
address = 'Budapest'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

47.4983815 19.0404707


Let's construct the good old API URI.

In [64]:
# 14,5 km will suffice, we are not making money in the suburbs anyways..
radius = 14500

# Gym will be our keyword because I want to keep it simple. I think we will have enough to work with.. especially with 1,5 km radius
search_query = 'Gym'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VPDI2D3XEB2I14NKL1EBV0RKHHK0Q4ZPPRJZIOXY2GFARGVV&client_secret=QJKHM15503UBRSR5FYC42D0EZOMSK5Q3WTIUVSSVKCMZHKLD&ll=47.4983815,19.0404707&v=20190625&query=Gym&radius=14500&limit=400'

### Get them results!

In [65]:
results = requests.get(url).json()

We've got several hits to work with, let's get to prettify this a little bit.

In [66]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,4d18963b6d103704db0b40bd,Márvány utca 17.,HU,Budapest,Magyarország,Alkotás utca,1262,"[Budapest, Márvány utca 17. (Alkotás utca), 11...","[{'label': 'display', 'lat': 47.49567290756484...",47.495673,19.024170,Krisztinaváros,1126,Budapest,Flex Gym,v-1561494453,NaN
1,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,5044d66de4b0a105081ee73e,Hajós u. 25,HU,NaN,Magyarország,NaN,1437,"[Hajós u. 25, Magyarország]","[{'label': 'display', 'lat': 47.50447149819039...",47.504471,19.057328,NaN,NaN,NaN,Super Gym,v-1561494453,NaN
2,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,56b1eb0b498e490552336e64,Arena Plaza,HU,Budapest,Magyarország,NaN,3764,"[Budapest, Arena Plaza, Magyarország]","[{'label': 'display', 'lat': 47.49817954772074...",47.498180,19.090520,NaN,NaN,Budapest,Peak Gym Aréna,v-1561494453,NaN
3,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,50f533dfe4b05f2e66f72902,Lomb u. 16,HU,Budapest,Magyarország,Frangepán u.,4739,"[Budapest, Lomb u. 16 (Frangepán u.), 1139, Ma...","[{'label': 'display', 'lat': 47.53606737028626...",47.536067,19.069795,NaN,1139,NaN,Gym City,v-1561494453,NaN
4,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,4f0c74ede4b04975989ab412,Katona József u.,HU,Budapest XIII. kerülete,Magyarország,NaN,1921,"[Budapest XIII. kerülete, Katona József u., Ma...","[{'label': 'display', 'lat': 47.51318136555004...",47.513181,19.053618,NaN,NaN,Budapest,Erőpont Gym,v-1561494453,NaN


We've learned how to process these kind of tables, so let's do it:

In [67]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Flex Gym,Gym / Fitness Center,Márvány utca 17.,HU,Budapest,Magyarország,Alkotás utca,1262,"[Budapest, Márvány utca 17. (Alkotás utca), 11...","[{'label': 'display', 'lat': 47.49567290756484...",47.495673,19.024170,Krisztinaváros,1126,Budapest,4d18963b6d103704db0b40bd
1,Super Gym,Gym / Fitness Center,Hajós u. 25,HU,NaN,Magyarország,NaN,1437,"[Hajós u. 25, Magyarország]","[{'label': 'display', 'lat': 47.50447149819039...",47.504471,19.057328,NaN,NaN,NaN,5044d66de4b0a105081ee73e
2,Peak Gym Aréna,Gym / Fitness Center,Arena Plaza,HU,Budapest,Magyarország,NaN,3764,"[Budapest, Arena Plaza, Magyarország]","[{'label': 'display', 'lat': 47.49817954772074...",47.498180,19.090520,NaN,NaN,Budapest,56b1eb0b498e490552336e64
3,Gym City,Gym / Fitness Center,Lomb u. 16,HU,Budapest,Magyarország,Frangepán u.,4739,"[Budapest, Lomb u. 16 (Frangepán u.), 1139, Ma...","[{'label': 'display', 'lat': 47.53606737028626...",47.536067,19.069795,NaN,1139,NaN,50f533dfe4b05f2e66f72902
4,Erőpont Gym,Gym,Katona József u.,HU,Budapest XIII. kerülete,Magyarország,NaN,1921,"[Budapest XIII. kerülete, Katona József u., Ma...","[{'label': 'display', 'lat': 47.51318136555004...",47.513181,19.053618,NaN,NaN,Budapest,4f0c74ede4b04975989ab412
5,SmartFit Gym,Gym,Király utca 8-10.,HU,Budapest,Magyarország,NaN,1215,"[Budapest, Király utca 8-10., Magyarország]","[{'label': 'display', 'lat': 47.49845698740046...",47.498457,19.056631,Deák Ferenc tér,NaN,Budapest,565049a2498e99c90a1e0489
6,Gym,Gym,Hess András tér 1-2,HU,Budapest,Magyarország,Hilton Budapest,705,"[Budapest, Hess András tér 1-2 (Hilton Budapes...","[{'label': 'display', 'lat': 47.50262712620703...",47.502627,19.033502,NaN,1014,Budapest,51276c48e4b0e675bc42bdb8
7,Central Gym,Martial Arts Dojo,Kálmán Imre utca 15,HU,Budapest V. kerület,Magyarország,NaN,1324,"[Budapest V. kerület, Kálmán Imre utca 15, 105...","[{'label': 'display', 'lat': 47.50688353746894...",47.506884,19.052794,NaN,1054,Budapest,5787f458498e877016688df0
8,Palace gym,Gym,NaN,HU,NaN,Magyarország,NaN,181,[Magyarország],"[{'label': 'display', 'lat': 47.49773057496788...",47.497731,19.038256,NaN,NaN,NaN,5392f8db498e0e408d346c67
9,In Balance Gym and Spa,Gym,Novotel,HU,Budapest,Magyarország,Rákóczi Utca 43-45,2208,"[Budapest, Novotel (Rákóczi Utca 43-45), 1088,...","[{'label': 'display', 'lat': 47.49705813556206...",47.497058,19.069767,NaN,1088,Budapest,4e88c7dce5fa8735be46204e


In [59]:
# Checking shape:
dataframe_filtered.shape

(50, 16)

Well 50 is not really much, but not bad either. We can work with this.

In [136]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# add the Gims as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue'
    ).add_to(venues_map)

# display map
venues_map

### Well, we can clearly see that on Buda, around the Gellérthegy, we are not very fit, because we lack Gyms. So maybe we could start there.